In [1]:
# Importa as bibliotecas e arquivos necessarios

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas
import os
from network_architectures import dense_autoencoder
import matplotlib.pyplot as plt

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

%load_ext tensorboard
!rm -rf ./logs/ 

In [2]:
#   Faz o download do fashion_mnist da base de dados do keras
mnist = keras.datasets.mnist

#   Carrega esse dataset, ja separando entre dados de treinamento e dados de teste
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#   Podemos verificar o formato desses arquivos
train_images.shape
test_images.shape
train_labels.shape
test_labels.shape

#   Segunda parte - processar os dados -----------------------------------------
#   Em seguida, precisamos processar esses dados. Nesse caso, o processamento se resume a
#   limitar o valor de cada pixel das imagens no intervalo [0,1]. Fazemos isso dividindo por 255
train_images = train_images / 255.0
test_images = test_images / 255.0

In [3]:
#   Terceira parte - Definir o modelo e fazer o treinamento --------------------

#   Carregamos a arquitetura do nosso modelo do nosso arquivo de arquiteturas. Isso deixa o codigo
#   bem modular e limpo
model = dense_autoencoder()

In [4]:
#   Compilamos o nosso modelo. Compilar significa dizer qual a funcao custo e otimizador vamos utilizar (entre outras coisinhas)

#   Nesse caso, o otimizador eh o "Adam". A funcao custo eh uma funcao mse
model.compile(optimizer='adam',
              loss='mse')

In [5]:
#Resumo do modelo
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               640       
_________________________________________________________________
dense_3 (Dense)              (None, 784)               101136    
_________________________________________________________________
reshape (Reshape)            (None, 28, 28)            0         
Total params: 202,772
Trainable params: 202,772
Non-trainable params: 0
__________________________________________________

In [6]:
#   Dessa vez, antes de proseguirmos com o treinamento, vamos criar um callback.
#   O callback serve para que o keras faca coisas entre epocas, como por exemplo,
#   salvar o melhor modelo.

In [7]:
#   um learning rate scheduler muda o learning rate de acordo com qual a epoca atual
#   nosso learning scheduler simplesmente muda a taxa ao passar de 10 epocas

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01,
    decay_steps=10,
    decay_rate=0.9,
    staircase=True)

learning_schedule = keras.callbacks.LearningRateScheduler(lr_schedule)

In [8]:
# Este callback faz com que o treinamento acabe mais rapidamente, caso a função custo não esteja mais sendo minimizada

earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [9]:
#   Para salvarmos a melhor epoca, usamos o callback de ModelCheckpoint
model_checkpoint = keras.callbacks.ModelCheckpoint('model_checkpoint.hdf5',
                    monitor='val_loss',
                    verbose=2,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='auto')

In [10]:
#   Para vizualizarmos melhor o treinamento do modelo, usaremos o Tensorboard
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [11]:
#   Para vizualizarmos as saídas da rede durante o treinamento, vamos criar criar um novo callback
#   Esse callback salva as imagens

n = 10

class VizualizarSaidasCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        figure, axis = plt.subplots(n, 4)
        plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
        
        for i in range(n):
            axis[i,0].imshow(train_images[i], cmap='gray')
            prediction = np.squeeze(model.predict(np.expand_dims(train_images[i], 0)))
            axis[i,1].imshow(prediction, cmap='gray')
            
            axis[i,2].imshow(test_images[i], cmap='gray')
            prediction = np.squeeze(model.predict(np.expand_dims(test_images[i], 0)))
            axis[i,3].imshow(prediction, cmap='gray')

            for k in range(4):
                axis[i,k].set_yticklabels([])
                axis[i,k].set_xticklabels([])
                axis[i,k].set_xticks([])
                axis[i,k].set_yticks([])        
     
        plt.tick_params(left=False, bottom=False)
        plt.savefig(os.path.join('train_visualization','fig_'+str(epoch)))
        plt.close()

In [12]:
#   criamos o callback para passar para o metodo .fit
callback = [learning_schedule, model_checkpoint, tensorboard, earlystop, VizualizarSaidasCallback()]

In [13]:
#   Agora vamos enfim treinar o modelo. Usamos o metodo "fit", passando como parametro nossos dados de treino, teste, e por quantas epocas
#   Esse metodo retorna os dados mostrados durante o treinamento, e em geral e interessante salva-los.

fit_history = model.fit(train_images, train_images, validation_data=(test_images, test_images), epochs=100, batch_size=64, callbacks=callback)

Epoch 1/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0638 - val_loss: 0.0409

Epoch 00001: val_loss improved from inf to 0.04086, saving model to model_checkpoint.hdf5
Epoch 2/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0404 - val_loss: 0.0393

Epoch 00002: val_loss improved from 0.04086 to 0.03925, saving model to model_checkpoint.hdf5
Epoch 3/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0388 - val_loss: 0.0380

Epoch 00003: val_loss improved from 0.03925 to 0.03799, saving model to model_checkpoint.hdf5
Epoch 4/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0381 - val_loss: 0.0380

Epoch 00004: val_loss did not improve from 0.03799
Epoch 5/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0377 - val_loss: 0.0373

Epoch 00005: val_loss improved from 0.03799 to 0.03735, saving model to model_checkpoint.hdf5
Epoch 6/100
938/938 [==============================] - 2s 3ms/step 

Epoch 48/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0349 - val_loss: 0.0353

Epoch 00048: val_loss did not improve from 0.03520
Epoch 49/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0348 - val_loss: 0.0356

Epoch 00049: val_loss did not improve from 0.03520
Epoch 50/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0350 - val_loss: 0.0355

Epoch 00050: val_loss did not improve from 0.03520
Epoch 51/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0348 - val_loss: 0.0351

Epoch 00051: val_loss improved from 0.03520 to 0.03509, saving model to model_checkpoint.hdf5
Epoch 52/100
938/938 [==============================] - 2s 3ms/step - loss: 0.0347 - val_loss: 0.0354

Epoch 00052: val_loss did not improve from 0.03509
Epoch 53/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0348 - val_loss: 0.0354

Epoch 00053: val_loss did not improve from 0.03509
Epoch 54/100
938/938 [===========


Epoch 00099: val_loss did not improve from 0.03486
Epoch 100/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0341 - val_loss: 0.0348

Epoch 00100: val_loss improved from 0.03486 to 0.03479, saving model to model_checkpoint.hdf5


In [14]:
# Voce pode vizualizar diversor gráficos e estatísticas sobre o treinamento usando o tensorboard

%tensorboard --logdir logs

In [15]:
#   Quarta parte - Salvar o modelo e outros dados --------------------
#   Neste ponto, nosso modelo ja esta treinado. Em geral, so queremos treinar o modelo uma unica vez, pois demora muito.
#   Vamos entao salvar esse modelo, junto com os dados de treinamento, no disco. Assim, se precisarmos usar ele novamente,
#   so precisamos carrega-lo.

In [16]:
#   Vamos primeiro salvar o historico de treinamento. Para isso, criaremos um dataframe no pandas (por ser mais facil de trabalhar assim)
fit_history_df = pandas.DataFrame(fit_history.history)

#   Com isso podemos salvar esses dados no disco diretamente
with open('fit_history.csv', mode='w') as f:
    fit_history_df.to_csv(f)
#   Reparar que um arquivo csv foi salvo no disco. Da uma olhadinha nesse arquivo depois

In [17]:
#   Em seguida, vamos salvar o modelo. Para isso, salvamos tanto a arquitetura quanto os pesos calculados.
#   A arquitetura do modelo podemos salvar como um arquivo json, com a funcao
model_json_string = model.to_json()
open('architecture.json', 'w').write(model_json_string)

2271

In [18]:
#   Eh importante notar que estamos salvando a ULTIMA EPOCA do modelo, e que ela nao necessariamente sera a melhor.
#   Para salvarmos a melhor epoca do modelo, precisamos configurar um "callback" no keras. Faremos isso em outro tutorial

#   Agora um arquivo json foi salvo. Da uma olhadinha nesse arquivo tbm
#   Ta na hora de salvar os pesos. Isso eh feito com a funcao
model.save_weights('model_weights.h5', overwrite=True)

In [19]:
#   Agora esse arquivo .h5 tbm foi salvo no disco. Com isso temos todo o nosso modelo salvo no disco, e nao precisaremos treina-lo novamente
#   Nosso arquivo de treinamento acaba por aqui. Agora vamos analisar o modelo no outro arquivo